In [24]:
import pandas as pd
import numpy as np

df_sb_list = pd.read_csv('C:\\Users\\Hi\\Desktop\\자료구조\\지하철역.csv')

# 1. 데이터 전처리

## 1) 기본

In [25]:
sb_list = df_sb_list.to_numpy()

## 2) 양방향 Graph 전처리(nan 존재x + 호선 구별o + 중복 호선 환승 고려x)

In [26]:
graph = {}

# 모든 역에 대해서 전처리
for i in range(5):
    for j in range(len(sb_list)):
        
        # graph 생성 - graph 마지막 원소 전이고, 비어있지 않은 역인 경우
        if(j < len(sb_list)-1 and not pd.isna(sb_list[j+1][i]) and not pd.isna(sb_list[j][i])):
            # graph에 해당 역 존재 : 해당 역의 key, value 빼서 다시 전처리 + 다음 역 추가시킨 graph 생성
            if (sb_list[j][i] + str(i+1)) in graph:
                keys = [k for k in graph[sb_list[j][i] + str(i+1)].keys()]
                values = [v for v in graph[sb_list[j][i] + str(i+1)].values()]
                
                dic = {}
                for d in range(len(keys)):
                    dic[keys[d]] = values[d]
                dic[sb_list[j+1][i] + str(i+1)] = 2
                graph[sb_list[j][i] + str(i+1)] = dic
                
            # graph에 해당 역 존재하지 않음 : 다음 역 추가시킨 graph 생성
            else:
                graph[sb_list[j][i] + str(i+1)] = {(sb_list[j+1][i] + str(i+1)):2}       
        # graph 생성 - 비어있는 역인 경우 : 건너뛰기
        elif(pd.isna(sb_list[j][i]) == True):
            pass
        # graph 생성 - 비어있는 역 아닌 경우
        else:
            # graph에 해당 역 존재 : 건너뛰기
            if (str(sb_list[j][i]) + str(i+1)) in graph:
                pass
            # graph에 해당 역 존재하지 않음 : 빈 원소로 두기
            else:
                graph[sb_list[j][i] + str(i+1)] = {}
                
        # graph 생성 - graph 첫 번째 원소 이후이고, 비어있지 않은 역인 경우
        if(0 < j and not pd.isna(sb_list[j-1][i]) and not pd.isna(sb_list[j][i])):
            # graph에 해당 역 존재 : 해당 역의 key, value 빼서 다시 전처리 + 다음 역 추가시킨 graph 생성
            if (sb_list[j][i] + str(i+1)) in graph:
                keys = [k for k in graph[sb_list[j][i] + str(i+1)].keys()]
                values = [v for v in graph[sb_list[j][i] + str(i+1)].values()]

                dic = {}
                for d in range(len(keys)):
                    dic[keys[d]] = values[d]
                dic[sb_list[j-1][i] + str(i+1)] = 2
                graph[sb_list[j][i] + str(i+1)] = dic

                '''if(sb_list[j][i] == '시청'): # 확인용 코드
                    print(keys)
                    print(graph[sb_list[j][i]])'''
            # graph에 해당 역 존재하지 않음 : 다음 역 추가시킨 graph 생성
            else:
                graph[sb_list[j][i] + str(i+1)] = {sb_list[j-1][i] + str(i+1):2}
        # graph 생성 - graph 첫번째 원소이거나, 비어있는 역인 경우 : 불가능한 경우
        else:
            pass
        
# print(graph) # 확인용 코드

## 3) 양방향 Graph 전처리 추가(중복 호선 환승 고려o)

In [27]:
graph_key = list(graph.keys())

for i in range(len(graph_key)):
    for j in range(i+1, len(graph_key)):
        if(graph_key[i][0:len(graph_key[i])-1] == graph_key[j][0:len(graph_key[j])-1]):
            keys = [k for k in graph[graph_key[i]]]
            values = [v for v in graph[graph_key[i]].values()]

            dic = {}
            for d in range(len(keys)):
                dic[keys[d]] = values[d]
            dic[graph_key[j]] = 2
            graph[graph_key[i]] = dic

            keys = [k for k in graph[graph_key[j]]]
            values = [v for v in graph[graph_key[j]].values()]

            dic = {}
            for d in range(len(keys)):
                dic[keys[d]] = values[d]
            dic[graph_key[i]] = 2
            graph[graph_key[j]] = dic

## 4) dijkstra 양방향 Graph 거리 계산(nan 존재x + 호선 구별o + 중복 호선 환승 고려o + 가독성 향상 + 경로 출력 + 환승 횟수 출력 + 하나의 dic으로 통일화 + 목적지까지 최소 거리 완성 시, break)

In [28]:
# 그래프 : 본인 노드 이름 : {연결된 노드 이름 : 연결된 노드까지의 거리}
# dist : 시작 노드에서 특정 노드까지 거리 + 경로 + 환승 횟수 저장 dic
# st : 시작 노드
# heap : 다음 특정 노드 저장
# plus dist : 특정 노드까지 최소 거리
# cur dist : 특정 노드까지 거리
# cur dest : 특정 노드 노드명

import heapq  # 우선순위 큐 구현을 위함

def dijkstra(graph, st, end):
    # 거리 + track + 환승횟수 초기 설정
    global dist
    dist = {node: [float('inf')] for node in graph} # [시작 노드에서 특정 노드까지 거리] : 무한대
    for node in graph:
        dist[node].append([]) # [시작 노드에서 특정 노드까지 경로] : 비어있음
        dist[node].append(0) # [시작 노드에서 특정 노드까지 환승 횟수] : 0
        
    dist[st][1] = [st]
    dist[st][0] = 0
    
    # heap 초기 설정
    queue = [] # [정보 저장 heap] 생성
    heapq.heappush(queue, [dist[st][0], st])  # [[시작 노드에서 시작 노드까지 거리], [시작 노드명]]를 [정보 저장 heap]에 저장

    # graph 생성
    while queue:  # heap에서 노드가 빌 때까지 반복
        cur_dist, cur_dest = heapq.heappop(queue) # [정보 저장 heap]에서 가장 작은 [[시작 노드에서 특정 노드까지 거리], [특정 노드명]] 빼기
        
        if(cur_dest == end): # 확인용 코드
            break
        
        if dist[cur_dest][0] < cur_dist: # [정보 저장 heap]에서 가장 작은 [시작 노드에서 특정 노드까지 거리] > [기존 시작 노드에서 특정 노드까지 거리]이면 건너띔
            print(dist[cur_dest][0], cur_dist) # 확인용 코드
            continue

        for new_dest, new_dist in graph[cur_dest].items(): # [특정 노드]에 연결된 [[특정 노드에서 다음 특정 노드까지 거리], [다음 특정 노드명]] 전부 돌기
            plus_dist = cur_dist + new_dist # [시작 노드에서 다음 특정 노드까지의 새로운 거리] = [정보 저장 heap]에서 가장 작은 [[시작 노드에서 특정 노드까지 거리], [특정 노드명]] + 특정 노드명에 연결된 [[특정 노드에서 다음 특정 노드까지 거리], [다음 특정 노드명]]
            if plus_dist < dist[new_dest][0]: # [시작 노드에서 다음 특정 노드까지 거리] > [시작 노드에서 다음 특정 노드까지의 새로운 거리]라면
                # 거리 최신화
                dist[new_dest][0] = plus_dist # [시작 노드에서 다음 특정 노드까지 거리] 최신화
                
                # 경로 최신화
                dist[new_dest][1] = dist[cur_dest][1][:] # [시작 노드에서 특정 노드까지 경로] 최신화 (특정 노드 이전까지 경로 얕은 복사 + 특정 노드도 경로에 추가)
                dist[new_dest][1].append(new_dest)
                    
                # 환승 횟수 최신화
                if(new_dest[0:len(new_dest)-1] == cur_dest[0:len(cur_dest)-1]): # [시작 노드에서 특정 노드까지 환승 횟수] 최신화 (특정 노드 이전까지 환승 횟수 + 특정 노드도 확인 후 추가)
                    dist[new_dest][2] = dist[cur_dest][2] + 1
                else:
                    dist[new_dest][2] = dist[cur_dest][2]
                heapq.heappush(queue, [plus_dist, new_dest]) # [[시작 노드에서 다음 특정 노드까지 거리], [다음 특정 노드명]]을 [정보 저장 heap]에 저장
    return dist

In [29]:
def print_dist_dijkstra():
    st = input("출발지 : ")
    end = input("도착지 : ")
    
    print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ[결과]ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")
    # print("배열 : ", dist[end]) # 확인용 코드
    print("거리 : ", dist[end][0], "분")
    print("경로 : ", dist[end][1])
    print("환승 횟수 : ", dist[end][2], "회")
    print("ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ")

In [30]:
print_dist_dijkstra()

출발지 : 시청1
도착지 : 상일동5
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ[결과]ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
거리 :  44 분
경로 :  ['시청1', '시청2', '을지로입구2', '을지로3가2', '을지로4가2', '동대문역사문화공원2', '신당2', '상왕십리2', '왕십리2', '왕십리5', '마장5', '답십리5', '장한평5', '군자5', '아차산5', '광나루5', '천호5', '강동5', '길동5', '굽은다리5', '명일5', '고덕5', '상일동5']
환승 횟수 :  2 회
ㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡㅡ
